In [1]:
import pathlib
import pandas as pd
import numpy as np
pd.reset_option
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [65]:
import lightgbm as lgb

In [3]:
from tqdm import tqdm
tqdm.pandas()
import pycountry
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from strsimpy.levenshtein import Levenshtein
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [1190]:
train = pd.read_csv("train.csv", index_col="pair_id")

In [1228]:
test = pd.read_csv("test.csv", index_col="pair_id")

In [1193]:

# Копии названий, нижний регистр
train["name_1_filtered"] = train["name_1"].str.lower()
train["name_2_filtered"] = train["name_2"].str.lower()


In [1229]:
test["name_1_filtered"] = test["name_1"].str.lower()
test["name_2_filtered"] = test["name_2"].str.lower()

In [1194]:
# Сущности в скобках
train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(r"\s+(\(.*\))"), " " )
train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(r"\s+(\(.*\))"), " " )


In [1230]:
test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(r"\s+(\(.*\))"), " " )
test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(r"\s+(\(.*\))"), " " )

In [1195]:
# Символы
train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(r"[^\w\s]"), "")
train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(r"[^\w\s]"), "")


In [1231]:
test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(r"[^\w\s]"), "")
test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(r"[^\w\s]"), "")

In [1196]:
# Short words
train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(r'\b[a-zA-Z0-9]{1}\b'), " ")
train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(r'\b[a-zA-Z0-9]{1}\b'), " ")

#### Entities

In [1197]:
legal={'aenp',
 'np',
 'ac',
 'as',
 'ab',
 'ace',
 'ad',
 'ae',
 'ag',
 'al',
 'amba',
 'ans',
 'apb',
 'aps',
 'as',
 'asa',
 'avv',
 'bv',
 'bpk',
 'bt',
 'bvba',
 'co',
 'cv',
 'ca',
 'cedel',
 'cin',
 'common code',
 'corp',
 'cusip',
 'cva',
 'cvoa',
 'de',
 'dd',
 'dno',
 'doo',
 'dba',
 'da',
 'ev',
 'ee',
 'eeg',
 'eirl',
 'elp',
 'eood',
 'epe',
 'eurl',
 'euroclear',
 'ext',
 'gbr',
 'gcv',
 'gesmbh',
 'gie',
 'gmbh',
 'hb',
 'hf',
 'is',
 'ibc',
 'id',
 'inc',
 'isin',
 'jtd',
 'kd',
 'kdd',
 'ks',
 'kas',
 'kb',
 'kd',
 'kda',
 'kft',
 'kg',
 'kgaa',
 'kk',
 'kkt',
 'kol',
 'kom',
 'ks',
 'kv',
 'ky',
 'lda',
 'ldc',
 'llc',
 'llp',
 'ltd',
 'ltda',
 'ltée',
 'na',
 'nt',
 'nv',
 'oe',
 'ohg',
 'ood',
 'oy',
 'oyj',
 'oü',
 'pl',
 'pc',
 'plc',
 'pma',
 'pmdn',
 'prc',
 'prp',
 'pt',
 'pty',
 'pvt',
 'ras',
 'ric',
 'rt',
 'sderl',
 'de',
 'en',
 'rl',
 'senc',
 'sennc',
 'nc',
 'sa',
 'saica',
 'sac',
 'sc',
 'scs',
 'sadecv',
 'safi',
 'sapa',
 'sarl',
 'sas',
 'sc',
 'sca',
 'sro',
 'scp',
 'scs',
 'sdn',
 'bhd',
 'security identifiers',
 'sedol',
 'senc',
 'sgps',
 'sic',
 'sicc',
 'sicovam',
 'sk',
 'snc',
 'soparfi',
 'sp',
 'zoo',
 'spa',
 'spol',
 'sro',
 'sprl',
 'scrl',
 'srk',
 'srl',
 'svm',
 'td',
 'tls',
 'vos',
 'valoren',
 'veb',
 'vof',
 'wertpapier',
 'kennnummer',
 'kenn',
 'nummer',    
 'wkn',
 'wpk'}

In [1198]:

for entity in tqdm(legal):
    train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(fr"\b({entity})\b"), " ")
    train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(fr"\b({entity})\b"), " ")


100%|████████████████████████████████████████| 154/154 [06:47<00:00,  2.64s/it]


In [1232]:
for entity in tqdm(legal):
    test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(fr"\b({entity})\b"), " ")
    test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(fr"\b({entity})\b"), " ")

100%|████████████████████████████████████████| 154/154 [02:54<00:00,  1.13s/it]


#### Countries

In [1202]:
countries = [country.name.lower() for country in pycountry.countries]
for country in tqdm(countries):
    train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(country), " ")
    train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(country), " ")


100%|████████████████████████████████████████| 249/249 [08:08<00:00,  1.96s/it]


In [1235]:
for country in tqdm(countries):
    test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(country), " ")
    test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(country), " ")

100%|████████████████████████████████████████| 249/249 [03:33<00:00,  1.16it/s]


#### StopWords

In [1203]:
def clean_stopwords(name):
    clear=''
    for word in name.split():
        #print(word)
        try:
            if fully_indexed[word]>0.45:
                clear=clear+word+' '          
                
        except:
            clear=clear+word+' '
    if clear=='':
        clear=name
    return clear

In [1206]:
stop_words=['international',  'industries', 'trading', 'logistics', 'rubber', 'private', 'llc', 'products', 'imp' ,'industrial', 'exp',\
            'shanghai','and','chemical','enterprises', 'plastic', 'chemicals', 'sports', 'group' ,'industria' ,'global', 'usa', 'dongguan', 'trade',\
            'plastics','technology', 'polymers','america', 'qingdao', 'shenzhen', 'services','textile','brasil','industry',\
            'logistica', 'materials', 'eercio', 'technologies','industrias', 'enterprise', 'guangzhou', 'electronics' ,'mfg', 'industries',\
            'industriales', 'sociedad', 'responsabilidad' ,'limitada' , 'north', 'west', 'south', 'east', 'solutions', 'systems', 'exploration',\
           'limited', 'transport', 'tire', 'bank', 'corporation', 'mfg', 'rubber']
for entity in tqdm(stop_words):
    train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")
    train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")

100%|██████████████████████████████████████████| 62/62 [02:29<00:00,  2.42s/it]


In [1236]:
for entity in tqdm(stop_words):
    test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")
    test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")

100%|██████████████████████████████████████████| 62/62 [01:06<00:00,  1.07s/it]


#### Russian words

In [1209]:
from transliterate import translit

In [1210]:
train[(train['name_1'].str.contains('[а-яА-Я]+', regex=True)|train['name_2'].str.contains('[а-яА-Я]+', regex=True))&train['is_duplicate']==1]

,name_1,name_2,is_duplicate,name_1_filtered,name_2_filtered
pair_id,,,,,
3372,ООО Химинвест групп,"ХИМИНВЕСТ ГРУПП, ООО",1,ооо химинвест групп,химинвест групп ооо
5118,"ООО""ИМПОРТ МОДА""","ООО""ИМПОРТ МОДА""",1,оооимпорт мода,оооимпорт мода
5451,SOUDAL NV,"ООО ""СОУДАЛ""",1,soudal,ооо соудал
12161,"ООО""ГУЧЧИ РУС""","ООО""ГУЧЧИ РУС""",1,ооогуччи рус,ооогуччи рус
16802,BOFER INSAAT ALUMINIUM SAN. VE TIC. LTD. STI,"ООО""БОФФЕР""",1,bofer insaat aluminium san ve tic sti,оообоффер
...,...,...,...,...,...
479648,"ООО ""КЬЮ ЭКСПРЕСС""","ООО ""КЬЮ ЭКСПРЕСС""",1,ооо кью экспресс,ооо кью экспресс
481656,"ООО ""БРААС-ДСК 1""","ООО ""БРААС-ДСК 1""",1,ооо браасдск 1,ооо браасдск 1
481759,GOTER S.R.L.,GOTER (ИТАЛИЯ),1,goter,goter


In [1211]:
russian_words=['ооо', 'зао' ]
for entity in tqdm(russian_words):
    train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(fr"{entity}"), " ")
    train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(fr"{entity}"), " ")

100%|████████████████████████████████████████████| 2/2 [00:03<00:00,  1.97s/it]


In [1212]:
russian_words=['ао', 'сп', ]
for entity in tqdm(russian_words):
    train["name_1_filtered"]=train["name_1_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")
    train["name_2_filtered"]=train["name_2_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")

100%|████████████████████████████████████████████| 2/2 [00:05<00:00,  2.60s/it]


In [1213]:
train.loc[train['name_2_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_2_filtered']=train.loc[train['name_2_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_2_filtered'].apply(lambda r:translit(r,'ru', reversed=True))

In [1214]:
train.loc[train['name_1_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_1_filtered']=train.loc[train['name_1_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_1_filtered'].apply(lambda r:translit(r,'ru', reversed=True))

In [1287]:
for entity in tqdm(russian_words):
    test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(fr"{entity}"), " ")
    test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(fr"{entity}"), " ")

100%|████████████████████████████████████████████| 2/2 [00:01<00:00,  1.17it/s]


In [1288]:
for entity in tqdm(russian_words):
    test["name_1_filtered"]=test["name_1_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")
    test["name_2_filtered"]=test["name_2_filtered"].replace(re.compile(fr"\b{entity}\b"), " ")

100%|████████████████████████████████████████████| 2/2 [00:02<00:00,  1.12s/it]


In [1289]:
test.loc[test['name_2_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_2_filtered']=test.loc[test['name_2_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_2_filtered'].apply(lambda r:translit(r,'ru', reversed=True))

In [1290]:
test.loc[test['name_1_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_1_filtered']=test.loc[test['name_1_filtered'].str.contains('[а-яА-Я]+', regex=True), 'name_1_filtered'].apply(lambda r:translit(r,'ru', reversed=True))

#### FillNA

In [1218]:
train.loc[train['name_2_filtered'].isnull(),['name_2_filtered']]=train.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")
train.loc[train['name_1_filtered'].isnull(),['name_1_filtered']]=train.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1219]:
train.loc[train.name_1_filtered.str.isspace(), ['name_1_filtered']]=train.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")
train.loc[train.name_2_filtered.str.isspace(), ['name_2_filtered']]=train.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1220]:
train.loc[train['name_2_filtered']=='',['name_2_filtered']]=train.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")
train.loc[train['name_1_filtered']=='',['name_1_filtered']]=train.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1227]:
train.to_csv('train_filtered.csv')

In [1296]:
test.loc[test['name_2_filtered'].isnull(),['name_2_filtered']]=test.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")
test.loc[test['name_1_filtered'].isnull(),['name_1_filtered']]=test.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1297]:
test.loc[test.name_1_filtered.str.isspace(), ['name_1_filtered']]=test.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")
test.loc[test.name_2_filtered.str.isspace(), ['name_2_filtered']]=test.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1298]:
test.loc[test['name_2_filtered']=='',['name_2_filtered']]=test.name_2.str.lower().replace(re.compile(r"[^\w\s]"), "")
test.loc[test['name_1_filtered']=='',['name_1_filtered']]=test.name_1.str.lower().replace(re.compile(r"[^\w\s]"), "")

In [1302]:
test.to_csv('test_filtered.csv')

In [1303]:
test

,name_1,name_2,name_1_filtered,name_2_filtered
pair_id,,,,
1,Sun Blinds Decoration Inc.,Indl De Cuautitlan Sa Cv,sun blinds decoration,indl cuautitlan
2,Eih Ltd.,"Dongguan Wei Shi Plastic Product Co., Ltd.",eih,wei shi product
3,Jsh Ltd. (Hk),Arab Shipbuilding And Repair Yard C,jsh,arab shipbuilding repair yard c
4,Better Industrial Ltd.,Farmacap Industria E Comercio Ltda,better,farmacap e comercio
5,Equipos Inoxidables Del Norte Sa De Cv,Bel Inc.,equipos inoxidables del norte,bel
...,...,...,...,...
213245,Dyna Logistics Inc.,Jiangsu Easero Logistics Ltd.,dyna,jiangsu easero
213246,Dsi,Kavalani & Sons W.L.L,dsi,kavalani sons wll
213247,Saint Gobain Abrasives (Shanghai),Zarabi,saint gobain abrasives,zarabi


#### Embeddings

Загружаем fasttext эмбеддинги

In [697]:
test=pd.read_csv('test_distance.csv', index_col=0)

In [700]:
train=pd.read_csv('train_withgpe_distance.csv', index_col=0)

### Creating features

In [702]:
train['len_diff']=train.apply(lambda r: np.abs(len(str(r.name_1_filtered))-len(str(r.name_2_filtered))),axis=1)

In [703]:
test['len_diff']=test.apply(lambda r: np.abs(len(str(r.name_1_filtered))-len(str(r.name_2_filtered))),axis=1)

In [705]:
train['token_sort_ratio'] = train.progress_apply(lambda r: fuzz.token_sort_ratio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 497819/497819 [01:10<00:00, 7069.08it/s]


In [706]:
train['token_set_ratio'] = train.progress_apply(lambda r: fuzz.token_set_ratio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 497819/497819 [01:15<00:00, 6573.66it/s]


In [707]:
train['fuzz_qratio'] = train.progress_apply(lambda r: fuzz.QRatio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 497819/497819 [01:07<00:00, 7420.85it/s]


In [708]:
train['fuzz_WRatio'] = train.progress_apply(lambda r: fuzz.WRatio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 497819/497819 [04:41<00:00, 1769.06it/s]


In [709]:
train['fuzz_partial_ratio'] = train.progress_apply(lambda r: fuzz.partial_ratio(str(r.name_1_filtered), str(r.name_2_filtered)), axis=1)

100%|████████████████████████████████| 497819/497819 [02:46<00:00, 2991.73it/s]


In [710]:
test['token_sort_ratio'] = test.progress_apply(lambda r: fuzz.token_sort_ratio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 213249/213249 [00:29<00:00, 7188.89it/s]


In [711]:
test['token_set_ratio'] = test.progress_apply(lambda r: fuzz.token_set_ratio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 213249/213249 [00:32<00:00, 6657.20it/s]


In [712]:
test['fuzz_qratio'] = test.progress_apply(lambda r: fuzz.QRatio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 213249/213249 [00:28<00:00, 7590.12it/s]


In [713]:
test['fuzz_WRatio'] = test.progress_apply(lambda r: fuzz.WRatio(r.name_1_filtered, r.name_2_filtered), axis=1)

100%|████████████████████████████████| 213249/213249 [01:55<00:00, 1846.97it/s]


In [714]:
test['fuzz_partial_ratio'] = test.progress_apply(lambda r: fuzz.partial_ratio(str(r.name_1_filtered), str(r.name_2_filtered)), axis=1)

100%|████████████████████████████████| 213249/213249 [01:12<00:00, 2940.43it/s]


In [715]:
levenshtein = Levenshtein()

train["levenstein"] = train.progress_apply(lambda r: levenshtein.distance(str(r.name_1_filtered), str(r.name_2_filtered)), axis=1)


100%|████████████████████████████████| 497819/497819 [01:54<00:00, 4332.68it/s]


In [716]:
normalized_levenshtein = NormalizedLevenshtein()

train["norm_levenstein"] = train.progress_apply(lambda r: normalized_levenshtein.distance(str(r.name_1_filtered), str(r.name_2_filtered)),
                                                axis=1)


100%|████████████████████████████████| 497819/497819 [01:56<00:00, 4261.72it/s]


In [717]:
test["levenstein"] = test.progress_apply(lambda r: levenshtein.distance(str(r.name_1_filtered), str(r.name_2_filtered)), axis=1)


100%|████████████████████████████████| 213249/213249 [00:50<00:00, 4209.51it/s]


In [718]:
test["norm_levenstein"] = test.progress_apply(lambda r: normalized_levenshtein.distance(str(r.name_1_filtered), str(r.name_2_filtered)),
                                                axis=1)

100%|████████████████████████████████| 213249/213249 [00:51<00:00, 4156.99it/s]


In [719]:
train.loc[(train.jaccard==0)&(train.is_duplicate==0),['is_duplicate']]=1

#### Train test split

In [359]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [720]:
scale_fts=['cosine', 'cityblock', 'jaccard', 'canberra', 'minkowski', 'braycurtis', 'len_diff', 'token_sort_ratio', 'token_set_ratio', 'fuzz_qratio','fuzz_WRatio', 'fuzz_partial_ratio', 'levenstein', 'norm_levenstein']

In [721]:
scaler = StandardScaler()
x_scaled=train.copy()
x_scaled[scale_fts] = scaler.fit_transform(x_scaled[scale_fts])

In [902]:
x_train,x_val,y_train,y_val=train_test_split(train[scale_fts], train["is_duplicate"], test_size=0.3, random_state=43 )

In [903]:
x_train

,cosine,cityblock,canberra,minkowski,braycurtis,len_diff,token_sort_ratio,token_set_ratio,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,levenstein,norm_levenstein
pair_id,,,,,,,,,,,,,
160168,0.621268,12.365551,200.477453,0.865528,0.711856,11,6,6,6,6,24,24.0,0.857143
384527,0.437474,7.572466,169.623205,0.557237,0.531537,2,60,60,60,60,68,16.0,0.444444
157383,0.910557,16.382311,213.193833,1.212546,0.939052,7,22,22,26,31,60,12.0,0.705882
432296,0.588565,12.333176,191.789482,0.985257,0.678589,2,44,44,44,60,50,5.0,0.625000
225332,0.867728,16.495438,205.980066,1.182875,0.880913,13,13,13,13,30,57,15.0,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387350,0.896252,16.519239,216.107715,1.198846,0.928376,13,42,42,30,60,75,15.0,0.714286
40754,0.766643,14.295923,209.643398,1.032149,0.841431,24,19,19,19,68,50,25.0,0.833333
248064,0.791392,15.299028,209.281977,1.113839,0.840683,17,35,35,32,60,50,20.0,0.800000


In [723]:
test_scale=test.copy()
test_scale[scale_fts] = scaler.transform(test_scale[scale_fts])

In [276]:
import catboost

In [837]:
model_cat = CatBoostClassifier(auto_class_weights='Balanced', silent=True)

In [838]:
{'reg_lambda': 0.1, 'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.2}

{'reg_lambda': 0.1, 'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.2}

In [839]:
FTS=['cosine', 'cityblock',  'canberra', 'minkowski', 'braycurtis', 'len_diff', 'token_sort_ratio', 'token_set_ratio', 'fuzz_qratio','fuzz_WRatio', 'fuzz_partial_ratio', 'levenstein', 'norm_levenstein']

In [840]:
model_cat.fit(train_pca, np.array(y_train))

In [856]:
tr_preds = model_cat.predict_proba(train_pca)
cv_preds = model_cat.predict_proba(val_pca)

In [857]:
THR=0.9

In [858]:
tr_preds = (tr_preds[:, 1] > THR).astype(np.int)
cv_preds = (cv_preds[:, 1] > THR).astype(np.int)

In [859]:
print(f1_score(y_train, tr_preds))
print(f1_score(y_val, cv_preds))

0.8875819190052091
0.7257866775643645


In [836]:
model_cat.feature_importances_

array([27.60432038, 10.74296968, 17.01305322, 14.8746829 , 16.50023118,
       13.26474263])

In [952]:
test[FTS]

,cosine,token_sort_ratio,cityblock,canberra
pair_id,,,,
1,0.906020,22,13.935598,217.988107
2,0.641881,22,13.360432,197.300345
3,0.798842,12,14.302099,213.492766
4,0.935938,16,16.232773,225.509049
5,0.654095,12,13.772058,204.566007
...,...,...,...,...
213245,0.886114,11,15.538820,205.093185
213246,0.899231,10,14.961913,213.284294
213247,0.804740,21,14.895469,213.103909


In [970]:
test_preds=model_cat.predict_proba(test[FTS].values)


In [971]:
test_preds = (test_preds[:, 1] >0.94).astype(np.int)

In [972]:
answer=pd.DataFrame(data=test_preds, index=test.index, columns=['is_duplicate']  )

In [973]:
answer['is_duplicate'].value_counts()

0    211748
1      1501
Name: is_duplicate, dtype: int64

In [975]:
answer['name_1']=test['name_1']

In [976]:
answer['name_2']=test['name_2']

In [979]:
answer[answer.is_duplicate==1].sample(30)

,is_duplicate,name_1,name_2
pair_id,,,
27560,1,Polymer Molded Products Llc.,Polymer Products Imp. & Exp. (Pvt) Ltd.
65058,1,Cauchos Vulcanizados De Colombia,Cauchos Vulcanizados Colombia Ltda
58808,1,Famotron Comercio De Equipmentos Te,Famotron Comercio De Equipamentos
176108,1,To The Order Of Modhumoti Bank,To The Order Of Kcb Bank Kenya Ltd.
33691,1,Ceat Kelani Radials (Pvt) Ltd.,Ceat Kelani International Tyres
62357,1,Rossini Do Brasil Industria E Comercio De Plas...,Fcc Industria E Comercio Ltda
72050,1,"Clear Freight, Inc.",Harbor Freight Tools
120358,1,Pt Lotus Indah Textile Industries,Lotus Indah Textile Industries
85120,1,"Sojitz (Shanghai) Co., Ltd.",Sojitz Mexicana Sa De Cv


In [974]:
answer.to_csv('answer.csv')